In [ ]:
import zipfile as zp
import pandas as pd
from pypac import PACSession as Session #or use requests below if non-ONS
#from requests import Session
from io import BytesIO
import os
import matplotlib.pyplot as plt

%matplotlib inline

# Creating Summary Prescription Data

This code has been created to work with 2016, 2017 and 2018 data, earlier data may have format inconsistencies that need to be dealt with.

# Set File Location of Prescribing Data

These files are the "GP practice prescribing data - Presentation level" zip files that come from: https://data.gov.uk/dataset/176ae264-2484-4afe-a297-d51798eb8228/gp-practice-prescribing-data-presentation-level

In [ ]:
# Path to zip files
path = r"C:\Users\[your username]\Loneliness\Raw_Data\\"

# Read in drug data

In [ ]:
# Get drug data (NB some drugs duplicated for illnesses)
drug_data = pd.read_csv(path[:-10] + "drug_list.csv")

# Process Prescribing Data

This code iterates over the monthly prescribing data, ultimately producing an aggregate table.

Note, if you want to find prescribing for non-loneliness relatived diseases, all you need to do is provide a different set of drug_data and edit the code_loneliness function.

In [ ]:
# Function to find loneliness related prescribing
def code_loneliness(x):
    out = {}
    # coding by illness categories
    for illness in drug_data['illness'].unique():
        out[illness] = x['BNF NAME'].str.contains("|".join(drug_data[drug_data['illness'] == illness]['medication']),
                                                  case=False, 
                                                  regex=True).astype('int16')
    # Make dataframe
    out = pd.DataFrame(out)
    # Add loneliness related disease binary - avoids double counting some drugs.
    out['loneliness'] = x['BNF NAME'].str.contains("|".join(drug_data['medication'].unique()),
                                                   case = False, 
                                                   regex = True).astype('int16')
    # Return dataframe multiplied by counts of items.
    return out.multiply(x['ITEMS'], axis=0)

In [ ]:
# Make dictionary for aggregation
agg_cols = {col : 'sum' for col in drug_data['illness'].unique()}
agg_cols['ITEMS'] = 'sum'
agg_cols['loneliness'] = 'sum'
for key in ['Date','SHA','PCT','pcstrip','CenterName','Street','Town','Town2','Postcode']:
    agg_cols[key] = 'first'

In [ ]:
monthly_data = []

for file in os.listdir(path):
    with zp.ZipFile(path + file) as zipf:
        zip_names = zipf.namelist()

        # Deal with Address Files
        addr_name = next((filename for filename in zip_names if "ADDR" in filename), None)
        # Open address file in pandas, set header.
        addr = pd.read_csv(zipf.open(addr_name), 
                           header=0, 
                           names = ["Date", "PracCode", "PracName","CenterName",
                                    "Street", "Town", "Town2", "Postcode"], 
                           usecols = range(8))

        # Deal with prescription info
        prescribe_name = next((filename for filename in zip_names if "PDPI" in filename), None)
        # Open prescribing files in pandas.
        prescribe = pd.read_csv(zipf.open(prescribe_name))
        prescribe.columns = prescribe.columns.str.strip()
        # Get counts of prescribing dataframe for loneliness related diseases
        loneliness_prescribing = code_loneliness(prescribe[['BNF NAME','ITEMS']])
        # merge dataframes
        prescribe = prescribe.merge(loneliness_prescribing, left_index=True, right_index=True)
        del loneliness_prescribing
        
        # merge in address information
        prescribe = prescribe.merge(addr, left_on = 'PRACTICE', right_on = 'PracCode')
        del addr
        
        # Create uniform postcode field
        prescribe['pcstrip'] = prescribe['Postcode'].str.replace("\s","")
        
        # get a summary - grouping by PracCode
        summary = prescribe.groupby('PracCode', as_index=False).agg(agg_cols)
        del prescribe
        
        monthly_data.append(summary)
        print(file)

In [ ]:
# concatenate all the monthly data together.
data = pd.concat(monthly_data, ignore_index = True)

In [ ]:
# Save aggregated data
data.to_csv(path[:-10] + "processed_data.csv")

# Add Postcode Location

Postcode location is pulled in from the latest ONS NSPL (National Statistics Postcode Lookup). There is a guide to fields here: http://geoportal.statistics.gov.uk/datasets/0a404beab6f544be8fb72d0c2b12d62b

NB - If using an ONS laptop `pip install pypac`, if not comment pypac import above and use requests.

In [ ]:
data = pd.read_csv(path[:-10] + "processed_data.csv", index_col=0)

In [ ]:
# Read in postcode lookup data
# This is the persistent link to the latest ONS NSPL
postcode_url = "http://geoportal.statistics.gov.uk/datasets/055c2d8135ca4297a85d624bb68aefdb_0.csv"

with Session() as session:
    response = session.get(postcode_url, verify = False)

field_dtypes = {'objectid': 'int32', 'pcd':'str', 'pcd2': 'str', 'pcds':'str', 'dointr':'str','doterm':'str',
                'usertype':'int8','oseast1m': 'float', 'osnorth1m': 'float', 'osgrdind':'int8', 'lat':'float', 
                'long':'float', 'X':'float', 'Y':'float', 'imd': 'int8',
                'oa11':'str', 'cty': 'str', 'ced':'str', 'laua': 'str', 'ward': 'str', 'hlthau':'str',
                'ctry': 'str','pcon': 'str','eer': 'str','teclec': 'str','ttwa': 'str','pct': 'str','nuts': 'str',
                'park': 'str','lsoa11': 'str','msoa11': 'str','wz11': 'str','ccg': 'str','bua11': 'str',
                'buasd11': 'str','ru11ind': 'str','oac11': 'str','lep1': 'str','lep2': 'str','pfa': 'str',
                'ced': 'str','nhser': 'str','rgn': 'str','calncv': 'str','stp': 'str'}

pc = pd.read_csv(BytesIO(response.content), dtype = field_dtypes)    

In [ ]:
# create pcstrip for matching
pc['pcstrip'] = pc['pcd'].str.replace("\s","")

NB - here I'm joining 2011 LSOA, 2011 MSOA, Rural-Urban Indicator, Region (formally GOR), Local Authority Area (effectively district), and IMD score (NB separate for each country). However, you can add any of the geography codes available in the NSPL.

In [ ]:
data_temp = data.merge(pc[['pcstrip','oseast1m','osnrth1m','lsoa11','msoa11','ru11ind','rgn','laua','imd']], 
                       how = 'left',
                       on = 'pcstrip')

In [ ]:
# Check for missing postcodes
data_temp[data_temp['oseast1m'].isnull()]['pcstrip'].value_counts()

In [ ]:
# Clean Missing Postcodes - appear to be typos.
new_pcs = {'DL154SB': 'DL54SB', 'WR59QT':'WR52QT', 'DN18QN':'DN48QN', 'HU32AE':'HU34AE','GL10QD': 'GL13NN',
           'ME122TZ':'ME102TZ', 'BS378NG':'BS374NG', 'CV115PO':'CV115PQ', 'TW152EA':'TW153EA', 'EN24EJ':'EN80BX', 
           'YO302JS':'YO306JA','L62UN':'L67UN', 'NG698DB':'NG98DA', 'HA24ES':'HA14ES'}

data['pcstrip'] = data['pcstrip'].map(new_pcs).fillna(data['pcstrip'])

In [ ]:
# Merge data
data = data.merge(pc[['pcstrip','oseast1m','osnrth1m','lsoa11','msoa11','ru11ind','rgn','laua','imd']], 
                  how = 'left', 
                  on = 'pcstrip')

In [ ]:
# Save aggregated data
data.to_csv(path[:-10] + "processed_data_with_postcodes.csv")

## Check Postcodes 

Some Practice Codes have more than one postcode associated with them. Possible reasons for this are:
* Practices move to a new location.
* Practices are assigned a new postcode but don't physically move.
* Practice postcodes are wrongly entered at a particular wave and subsequently fixed.

There are 794 practices codes which have more than 1 postcode assigned to them, this is about 7% of unique practices.

764 for those practices have 2 postcodes associated with them, while 30 have 3 postcodes.

102 of these practices fall within the same LSOA, 669 fall within 2 different LSOAs, and 30 within 3 different LSOAs.

We'll ignore this for now - this will require some more advanced cleaning - useful to be aware of though.

In [ ]:
# Check for 1 postcode per Practice Code
pc_prac_counts = data.groupby('PracCode')['pcstrip'].unique().map(len)
# 794 practices have more than 1 postcode associated with it.
pc_prac_counts[pc_prac_counts > 1].count()

In [ ]:
# Practice codes with multiple associated postcodes account forc. 7% of the data
pc_prac_counts[pc_prac_counts > 1].count()/len(pc_prac_counts)

In [ ]:
pc_prac_counts[pc_prac_counts > 1].value_counts()

In [ ]:
# Count of these Practices falling within the same LSOA
(data[data['PracCode'].isin(pc_prac_counts[pc_prac_counts > 1].index)]
 .groupby('PracCode')['lsoa11']
 .unique()
 .map(len)
 .value_counts())

# Subsetting the Data

## Use only General Practice surgeries

Use the 'Patients Registered at a GP Practice' data from: https://digital.nhs.uk/data-and-information/publications/statistical/patients-registered-at-a-gp-practice to get GP surgery codes and subset the data.

In [ ]:
data = pd.read_csv(path[:-10] + "processed_data_with_postcodes.csv", index_col = 0)

In [ ]:
# Get GP files
gp_path = r"C:\Users\[your username]\Loneliness\GP Data\\"

In [ ]:
gp_combine = []

for file in os.listdir(gp_path):
    # read file into pandas
    month, year = file[-10:-4].split("-")
    
    # Deal with different file structures
    if (year == '16') or (year == '17' and month == 'jan'):
        gp_data = pd.read_csv(gp_path + file)
        gp_data['DATE'] = "01" + month.upper() + "20" + year
        gp_data.columns = gp_data.columns.str.upper()
        gp_data = gp_data.rename(columns = {'DATE':'Date','GP_PRACTICE_CODE':'PracCode','TOTAL_ALL':'NUMBER_OF_PATIENTS'})
        gp_combine.append(gp_data[['Date','PracCode','POSTCODE','NUMBER_OF_PATIENTS']])
    else:
        gp_data = pd.read_csv(gp_path + file)
        gp_data.columns = gp_data.columns.str.upper().str.replace(" ","_")
        gp_data = gp_data.rename(columns = {'EXTRACT_DATE':'Date', 'CODE':'PracCode'})
        if year == '17' and month == 'jun':
            gp_data['Date'] = "01" + month.upper() + "20" + year
        gp_combine.append(gp_data[['Date','PracCode','POSTCODE','NUMBER_OF_PATIENTS']])
    print(file)

In [ ]:
# concatenate all the gp data together.
gp_data = pd.concat(gp_combine, ignore_index = True)

In [ ]:
# Get the unique codes for GP surgeries and subset the prescribing data according to these codes.
gp_ids = gp_data['PracCode'].unique()
data = data[data['PracCode'].isin(gp_ids)].copy()
data.shape

In [ ]:
# Make date a datetime variable
gp_data['Date'] = pd.to_datetime(gp_data['Date'], format = '%d%b%Y')

In [ ]:
# Make date a datetime variable - days are assigned as first day of the month.
data['Date'] = pd.to_datetime(data['Date'], format = '%Y%m')

In [ ]:
# Merge on the basis of date and PracCode - produces some nulls for counts.
# It may be possible to predict missing values using a time-series model.
data = data.merge(gp_data, how = 'left', on = ['Date','PracCode'])

In [ ]:
# Save aggregated data
data.to_csv(path[:-10] + "processed_data_with_postcodes_GPs.csv")

# Generate Statistics from Prescribing Counts

## Percentages At Postcode Level

Aggregate observations to postcodes and compute percentages for 'depression', 'alzheimers', 'blood pressure', 'hypertension', 'diabetes', 'cardiovascular disease', 'insomnia', 'addiction', 'social anxiety', and 'loneliness'.

## Outlier Removal

Should we remove some GPs on the basis that they have very low/high values which might indicate they are not accessible to the general population, and instead represent specialist services?

Currently, we won't do this, but it's an advanced task to look into.

## Standardise Percentages

Should we standardise within time points, or across them? Or standardise with GP practices or across them?

Can't standardize within GPs, as can't then compare between GPs.

Can't standardise across GPs within years, as can't then compare between years.

Can't standardise across GPs across years, as can't then disambiguate changes to rank order over time.

<u> First Step </u>

Take the average percentage of disease groups within postcodes annually - this is then an annual summary measure of prescribing by postcode. Aggregation entire depends on desired time frame for analysis.

NB, this is a mean of percentages - could also calculate an overall percentage by summing monthly counts by year and dividing through by monthly sum of items.

<u> Second Step </u>

z-score standardise for earliest year observed across GPs. Store mean and standard deviation.

z-score standardise subsequent years wrt baseline mean and standard deviation.

OR

Use min-max normalisation by year (decile normalisation?). This standardises the different percentages to the same range - although in theory they're comparable anyway...

## Aggregation and Percentages

In [ ]:
data = pd.read_csv(path[:-10] + "processed_data_with_postcodes_GPs.csv", index_col = 0)

In [ ]:
# Make dictionary for aggregation
# counts to sum
agg_cols = {col : 'sum' for col in drug_data['illness'].unique()}
agg_cols['ITEMS'] = 'sum'
agg_cols['loneliness'] = 'sum'
agg_cols['NUMBER_OF_PATIENTS'] = 'sum'

# Other data to preserve
for key in ['SHA','PCT','Street','Town','Town2','Postcode','oseast1m', 'osnrth1m',
            'lsoa11', 'msoa11','ru11ind', 'rgn', 'laua', 'imd']:
    agg_cols[key] = 'first'

In [ ]:
# Do aggregation and produce counts by postcode by date.
data = data.groupby(['pcstrip','Date'], as_index=False).agg(agg_cols)

In [ ]:
# Generate percentages
perc_cols = drug_data['illness'].unique()
target_cols = perc_cols + '_perc'

# Percentages for discrete illness groups
data[target_cols] = data[perc_cols].divide(data['ITEMS'], axis=0) * 100

# Overall percentage for loneliness realted disease prescribing
data['loneliness_perc'] = data['loneliness'].divide(data['ITEMS'], axis=0) * 100

## Standardisation (z-scores)

In [ ]:
# Firstly aggregate percentages by postcodes by year.
data['Year'] = data['Date'].dt.year
# Aggregation
cols = {'NUMBER_OF_PATIENTS': 'mean', 'SHA':'first', 'PCT':'first', 'oseast1m':'first', 'osnrth1m':'first',
        'lsoa11': 'first', 'msoa11': 'first', 'ru11ind': 'first', 'rgn': 'first', 'laua':'first', 'imd': 'first',
        'depression_perc': 'mean', 'alzheimers_perc': 'mean', 'blood pressure_perc': 'mean', 'hypertension_perc': 'mean',
        'diabetes_perc': 'mean', 'cardiovascular disease_perc': 'mean', 'insomnia_perc': 'mean', 'addiction_perc': 'mean',
        'social anxiety_perc': 'mean', 'loneliness_perc': 'mean'}

data = data.groupby(['pcstrip','Year'], as_index=False).agg(cols)

In [ ]:
# The mean value returns a value broadly in the centre of the distribution of respective disease classes.
# Therefore we'll go with an un-truncated arithmetic mean.
# Can always revisit this assumption later.

per_cols = ['depression_perc', 'alzheimers_perc', 'blood pressure_perc', 'hypertension_perc', 
            'diabetes_perc', 'cardiovascular disease_perc', 'insomnia_perc', 'addiction_perc',
            'social anxiety_perc', 'loneliness_perc']

# Get mean and std for baseline (2016)
mean_std = data[data['Year'] == 2016][per_cols].agg(['mean','std'])

In [ ]:
# Make new column names.
std_cols = [col[:-4] + 'zscore' for col in per_cols]

zscores = []    
# z-score standardise for each year by baseline mean and std 
for year in [2016,2017,2018]:
    zscores.append((data.loc[data['Year'] == year, per_cols] - mean_std.loc['mean', per_cols]) / mean_std.loc['std', per_cols])

zscores = pd.concat(zscores).sort_index()
data[std_cols] = zscores

In [ ]:
# plot zscores for loneliness
f, [ax1, ax2, ax3] = plt.subplots(1,3, figsize = (14,6), sharey = True)

# Note that there appears to be increasing variation in lonelines prescribing over time.
# These means are comparable as standardised using 2016 mean and std.
data[data['Year'] == 2016]['loneliness_zscore'].hist(bins=100, ax = ax1)
data[data['Year'] == 2017]['loneliness_zscore'].hist(bins=100, ax = ax2)
data[data['Year'] == 2018]['loneliness_zscore'].hist(bins=100, ax = ax3);

In [ ]:
# Save aggregated data
data.to_csv(path[:-10] + "processed_data_with_postcodes_GPs_stats.csv")

# Create Loneliness Variable

The actual loneliness variable we work with is the sum of the standardised scores of: depression, alzheimers, hypertension, insomnia, addiction and social anxiety, for each year of interest.

This means that the loneliness variable is actually an equally weighted index of the above domains.

In [ ]:
# sum function ignores NAs
data['loneills'] =  data[['depression_zscore', 'alzheimers_zscore', 'hypertension_zscore', 'insomnia_zscore',
                          'addiction_zscore','social anxiety_zscore']].sum(axis=1)

In [ ]:
# plot zscores for loneills
f, [ax1, ax2, ax3] = plt.subplots(1,3, figsize = (14,6), sharey = True)

# Note that there appears to be increasing variation in lonelines prescribing over time.
data[data['Year'] == 2016]['loneills'].hist(bins=100, ax = ax1)
data[data['Year'] == 2017]['loneills'].hist(bins=100, ax = ax2)
data[data['Year'] == 2018]['loneills'].hist(bins=100, ax = ax3);

In [ ]:
# Save aggregated data
data.to_csv(path[:-10] + "final_data.csv")